# Slater Condon tools
Tools to evaluate the [Slater Condon rules](https://en.wikipedia.org/wiki/Slater%E2%80%93Condon_rules) for quantum determinants and multideterminants.

Tasks
- ~~etwo function for determinants~~
- ~~eone/etwo functions for multideterminants~~
- cross terms
- Correct energies for 
  - ~~cs singlet~~
  - ~~triplet~~
  - oss,
  - gvb
  - CISD
- Align function for determinants
  - Don't forget spin
- ~~Build simple closed/open determinants e.g. det("aabbccd") with alternating spins~~  

In [ ]:
# Imports and utilities:
import copy
import math
import itertools
from collections import defaultdict

def isclose(a,b,tol=1e-5): return abs(a-b)<tol
def pairs(L): 
  """
  Loops over pairs of items in a list:
  >>> list(pairs("ABC"))
  [('A', 'B'), ('A', 'C'), ('B', 'C')]
  """
  return itertools.combinations(L,2)


In [ ]:
class Fermion:
  """
  Create a simple object for fermions.
  >>> Fermion("a","up")
  a
  
  Equality is defined for Fermions:
  >>> Fermion("a","up")==Fermion("a","up")
  True

  You can also use integers as labels:
  >>> Fermion(1,"down")
  1̅
  """
  def __init__(self,label,spin):
    self.label = label
    self.spin = spin
    return
  
  def tostring(self): return ferm(self.label,self.spin)
  def __str__(self): return self.tostring()
  def __repr__(self): return self.tostring()
  def tuple(self): return self.label,self.spin
  def __hash__(self): return hash(self.tuple())
  def __eq__(self,other): return self.tuple()==other.tuple()
  def __lt__(self,other): return self.tuple()<other.tuple()

def ferm(label,spin,stype="bar"):
  """
  Label fermion spins with up/down arrows:
  >>> ferm("a","up",stype="arrow")
  'a↑'
  >>> ferm("a","down",stype="arrow")
  'a↓'
  >>> ferm("a","up",stype="bar")
  'a'
  >>> ferm("a","down",stype="bar")
  'a̅'
  >>> ferm("a","down")
  'a̅'
  """
  spins = dict(up="",down="\u0305") if stype=="bar" else dict(up="↑",down="↓")
  return "%s%s" % (label,spins[spin])


In [ ]:
class Det:
  """
  Create a simple object for antisymmetric slater determinants of fermions:
  >>> Det(Fermion("a","up"),Fermion("a","down"))
  |aa̅>
  >>> Det(Fermion("a","up"),Fermion("a","up")).zero()
  True
  >>> Det(Fermion("a","up"),Fermion("a","down")).eone()
  2haa

  Equality works:
  >>> Det(Fermion("a","up"),Fermion("a","down")) == det2(2)
  True

  There's a helper function `det` to construct simple determinants.
  >>> det("aa").etwo()
  Jaa
  >>> det("aabbc").etwo()
  Jaa + 4Jab - 2Kab + Jbb + 2Jac - Kac + 2Jbc - Kbc
  >>> det("aabb").energy()
  2haa + 2hbb + Jaa + 4Jab - 2Kab + Jbb

  Can also use the `det2` helper function, which takes nel, mult, and
  has flexible labels:
  >>> det2(2,mult=3).energy()
  haa + hbb + Jab - Kab
  >>> det2(2,mult=3,labeltype="numbers").energy()
  h11 + h22 + J12 - K12
  """
  def __init__(self,*fermions):
    self.fermions = fermions
    return
  
  def __hash__(self): return hash(self.fermions)
  def __eq__(self,other): return self.fermions==other.fermions
  def __lt__(self,other): return self.fermions<other.fermions
  def tostring(self): return "".join(["|"]+[str(f) for f in self.fermions]+[">"])
  def __repr__(self): return self.tostring()
  def zero(self): return (len(self.fermions) == 0) or (len(self.fermions) > len(set(self.fermions)))
  
  def eone(self): return EnergyExpression(*[h(f.label) for f in self.fermions])

  def etwo(self):
    E = EnergyExpression()
    nf = len(self.fermions)
    for i in range(nf):
      fi = self.fermions[i]
      for j in range(i+1):
        fj = self.fermions[j]
        li,lj = (fi.label,fj.label) if fi<fj else (fj.label,fi.label)
        if fi.spin == fj.spin:
          if li != lj:
            E.add(J(li,lj))
            E.add(K(li,lj),-1)
        else:
          E.add(J(li,lj))
    return E
  
  def energy(self): return self.eone() + self.etwo()

def det(labels): 
  """
  Convenience function to construct a determinant with alternating up/down spins.
  >>> det("aabbccd")
  |aa̅bb̅cc̅d>
  """
  return Det(*[Fermion(l,s) for (l,s) in zip(labels,itertools.cycle(["up","down"]))])

def det2(nel,mult=0,labeltype="letters"):
  """
  Alternate, more flexible way of constructing determinants. 
  Needs a better name.
  >>> det2(2)
  |aa̅>
  >>> det2(5,labeltype="numbers")
  |11̅22̅3>
  >>> det2(4,mult=3)
  |aa̅bc>
  """
  from string import ascii_letters
  labels = ascii_letters if labeltype == "letters" else list(range(1,200))
  if mult==0:
    mult = 1 if nel%2==0 else 2

  nopen = mult-1
  nclosed,ntest = divmod(nel-nopen,2)
  assert ntest == 0
  #nalpha,nbeta = nclosed+nopen,nclosed # if uhf is desired
  fermions = []
  for orb in range(nclosed):
    fermions.append(Fermion(labels[orb],"up"))
    fermions.append(Fermion(labels[orb],"down"))
  for orb in range(nclosed,nclosed+nopen):
   fermions.append(Fermion(labels[orb],"up"))
  return Det(*fermions)

def ndiff(d1,d2): 
  """
  Determine the number of fermions differing in two determinants:
  >>> ndiff(det("aab"),det("aac"))
  1
  """
  return len(set(d1.fermions).difference(set(d2.fermions)))

In [ ]:
class ExpressionDict:
  """
  Use a default dictionary to collect terms and coefficients
  to mock up symbolic like term evaluation.

  >>> ED = ExpressionDict("A","B")
  >>> ED.add("C")
  >>> ED.add("D",-1)
  >>> ED.scale(2)
  >>> ED
  2A + 2B + 2C - 2D
  """
  def __init__(self,*terms):
    self.terms = defaultdict(float)
    for term in terms:
      self.add(term)
    return

  def add(self,term,coef=1): self.terms[term] += coef  

  def add_scaled_terms(self,other,scale=1):
    for term,coef in other.terms.items():
      self.terms[term] += coef*scale
    return
    
  def scale(self,scale):
    for d in self.terms:
      self.terms[d] *= scale
    return

  def __repr__(self): return self.tostring()

  def __add__(self,other):
    E = copy.deepcopy(self)
    E.add_scaled_terms(other)
    return E
    
  def __iadd__(self,other): self.add_scaled_terms(other)

  def tostring(self):
    text = []
    for i,(d,c) in enumerate(self.terms.items()):
      if i>0 and c<0:
        text.append(" - ")
        text.append(ftos(-c))
      else:
        if i>0: text.append(" + ")
        text.append(ftos(c))
      text.append(str(d))
    return "".join(text)

def ftos(a,digs=5):
  """
  Helper function to display coefficients properly.
  
  Don't display coefficients of 1 (ie show 'J' rather than '1.0J')
  >>> ftos(1.0)
  ''

  If coefficients are close to an integer, show an integer: 
  >>> ftos(2.0)
  '2'

  Otherwise, round numbers to `digs` digits:
  >>> ftos(math.sqrt(2))
  '1.41421'
  """
  b = round(a,digs)
  bi,bf = divmod(b,1)
  if isclose(b,1): return ""   # Don't return 1's
  if isclose(bf,0): return str(int(bi)) # Return ints as ints
  return str(b) # Otherwise, round

In [ ]:
class MultiDet(ExpressionDict):
  """
  Define determinants for multiconfigurational wave functions. Uses 
  ExpressionDicts to keep track of term coefficients.
  
  Example: Define an open-shell singlet wave function
  >>> oss = MultiDet(det("aabc"),det("aacb"))
  >>> oss
  |aa̅bc̅> + |aa̅cb̅>
  >>> oss.normalize()
  >>> oss
  0.70711|aa̅bc̅> + 0.70711|aa̅cb̅>
  >>> oss.eone()
  2haa + hbb + hcc
  >>> oss.etwo()
  Jaa + 2Jab - Kab + 2Jac - Kac + Jbc
  >>> oss.energy()
  2haa + hbb + hcc + Jaa + 2Jab - Kab + 2Jac - Kac + Jbc
  """
  def norm(self): return sum(c*c for d,c in self.terms.items())
  def normalize(self): self.scale(1/math.sqrt(self.norm()))
  def eone(self):
    E = EnergyExpression()
    for det in self.terms: E.add_scaled_terms(det.eone(),self.terms[det]**2)
    # Add det-det cross-terms here, which are nonzero if dets differ by one fermion 
    return E

  def etwo(self):
    E = EnergyExpression()
    for det in self.terms: E.add_scaled_terms(det.etwo(),self.terms[det]**2)
    # Add det-det cross-terms here
    return E
  
  def energy(self): return self.eone() + self.etwo()


In [ ]:
class EnergyExpression(ExpressionDict):
  """
  Container to construct and manipulate energy expressions. Uses 
  ExpressionDicts to keep track of term coefficients.
  >>> EnergyExpression(h(1,1),J(1,2),K(1,2))
  h11 + J12 + K12
  """
  pass
    
# Templates for energy terms. Gives a single point to rewrite the notation:
def texpression(template):
  def f(i,j=False): 
    return template.format(i, j or i)
  return f

# or "h[%s,%s], J[%s,%s], K[%s,%s].
# or (ij|kl), <ij||kl>
# or latex expressions.
h = texpression("h{0}{1}") 
J = texpression("J{0}{1}")
K = texpression("K{0}{1}")


In [ ]:
import doctest; doctest.testmod()

TestResults(failed=0, attempted=39)